<a href="https://colab.research.google.com/github/restrepo/lens_dask/blob/master/lens_dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lens Dask

## Convert lens json to dask json

In [1]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir( "drive/My Drive/Lens/LAT" )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
import os
import re

In [ ]:
jz=os.listdir('data')

In [ ]:
jz=[j for j in jz if j.find('json.gz')>-1]

In [12]:
jz

[]

In [ ]:
for j in jz:
    ln=pd.read_json('data/'+j,compression='gzip')
    ln.to_json('data/'+j.split('.gz')[0],lines=True,orient='records')

See https://towardsdatascience.com/speeding-up-your-algorithms-part-4-dask-7c6ed79994ef

## Load as dask dataframes 

An error appears when trying to load a json file into a Dask DataFrame with the `blocksize` option with some specific number of records, in this example: 191 records. The error seem to not depend on some specific record, e.g the first one or the last one, or any subset as displayed below:

The solution is given here: https://stackoverflow.com/a/55027036/2268280

> You might consider increasing the size of the sample that Dask reads when determining metadata like column names.
```python
df = dd.read_json(..., sample=2**26)

In [1]:
import dask.dataframe as dd
import os
import re

In [2]:
os.getcwd()

'/home/restrepo/prog/medicion/lens_dask'

In [3]:
# must end with "/"
data_dir='data/'
if re.search('^/content',os.getcwd() ):
    data_dir='lines_True_orient_records/'

In [4]:
jz=os.listdir(data_dir)

In [5]:
jz=sorted( [j for j in jz if re.search('^lens_lat_.*json$',j)] )

In [6]:
good=[]
EXTRACT_GOOD=False
if EXTRACT_GOOD:
    ln=dd.read_json(data_dir+jz[0],blocksize=50e6,sample=2**26)
    for j in range(len(jz)):
        tmp=dd.read_json(data_dir+jz[j],blocksize=50e6,sample=2**26)
        try:
            kk=tmp.compute().shape[0]
            good.append(j)
            ln=ln.append( dd.read_json(data_dir+jz[j],blocksize=50e6,sample=2**26)  )
        except:
            pass

test dask tasks with good dataframes

In [7]:
if good:
    print(good)

In [8]:
good=range(47)[::-1]#[2, 3, 4, 5, 9, 10, 11, 12, 14, 15, 17, 20, 23, 26, 37, 42, 43, 45, 46, 47]

In [9]:
ln=dd.read_json(data_dir+jz[good[0]],blocksize=50e6,sample=2**26)

for i in good[1:6]:
    ln=ln.append( dd.read_json(data_dir+jz[good[i]],blocksize=50e6,sample=2**26)  )

In [10]:
ln.journal.compute().shape

(243034,)

### Does not improve upon pandas!

In [11]:
import pandas as pd
ln=pd.read_json(data_dir+jz[good[0]],lines=True,orient='records')

for i in good[1:6]:
    ln=ln.append( pd.read_json(data_dir+jz[good[i]],lines=True,orient='records')  )
ln.journal.shape    

(243034,)

for j in jz[1:]:
    ln=ln.append( dd.read_json('data/'+j ,blocksize=50e6) )

## Find the dask bug

In [ ]:
kk=ln[ln.authors.astype(str).str.contains('University of Antioquia')].reset_index(drop=True)

In [ ]:
kk.compute()

Bad

In [ ]:
import pandas as pd

In [ ]:
%%writefile drive.cfg
[FILES]
UDEA_authors_with_WOS_info.json       =1o1eVT4JD0FMMICq_oxrTJOzWh47veBMw
WOS_SCI_SCP_PTJ_CTR.json.gz           =19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
lens_lat_2019-04-01_2019-06-30.json.gz=11IM--JWIrFq6s2Bn-lNKtTAWm_XANAGx
lens_lat_2018-12-01_2019-03-31.json.gz=11D_TH-Jp95qcjf_5oLMiyZXto98eXFZv
lens_lat_2018-09-01_2018-11-30.json.gz=11CjOQb_qaz_Igg_BouNP1bayKvjMmfLM
lens_lat_2018-06-01_2018-08-31.json.gz=1191l19-rBlXWyV1FpkeGS60aMGGEhlY1
lens_lat_2018-02-01_2018-05-31.json.gz=117XflAqxJOtSqbNRuAD7qvdXKcq9sWV9
lens_lat_2017-12-01_2018-01-31.json.gz=11-gmcyfHjtZOhPSEzQGkgDBZO8NmWoUE
lens_lat_2017-06-01_2017-08-31.json.gz=11-JllgMrMw3_3zBH1rgGwxai97tjiobt
lens_lat_2017-02-01_2017-05-31.json.gz=10nQKCGer00evOusselh272ArLFEkdCDT
lens_lat_2016-12-01_2017-01-31.json.gz=10m_Fo6aDbjhN5cXpFHj73FXSghpn4OQ3
lens_lat_2016-09-01_2016-11-30.json.gz=10kbscWWxVDJbeuotDUU7e9IHRAiS59lO
lens_lat_2016-06-01_2016-08-31.json.gz=10hOv4cFEdE-8oC3310CTp0A0_SN-H5Qt
lens_lat_2016-03-01_2016-05-31.json.gz=10gIrPWa5OBMxjm_rLQ_jQ2Gu_M6-zGgD
lens_lat_2016-01-01_2016-02-29.json.gz=10dSIx1s-im-3ZYlmXgBCbhdUmC8BOzid
lens_lat_2015-11-01_2015-12-31.json.gz=10cWT0BDn2IlD9zBRflTxIOTdIcqZS-B4
lens_lat_2015-07-01_2015-10-31.json.gz=10ahPuzX2y70vtXqO9xLHkkeg7F5EOHnM
lens_lat_2015-03-01_2015-06-30.json.gz=10VtXIUJ3XOE7VyA9QCLL2H3yGP0DnaFw
lens_lat_2015-01-01_2015-02-28.json.gz=10TZfHu0fTlWLfopa-sq66Suzh35_2mkp
lens_lat_2014-11-01_2014-12-31.json.gz=10T5eHtSB9Gbmz5Ihtbj-xry8KxkVI3i5
lens_lat_2014-07-01_2014-10-31.json.gz=10O5EpnCR5Pn0WV-J8rIG46xP34bwUYyg
lens_lat_2014-03-01_2014-06-30.json.gz=10Di2UcHTlfsgYPfpzKHTgWgXQibSznTg
lens_lat_2014-01-01_2014-02-28.json.gz=1092RZhn-0oXve98bAecVULaHJmg9_2q5
lens_lat_2013-09-01_2013-12-31.json.gz=107cplrrWFZr__ysGQvmIIn2GoA43QEi7
lens_lat_2013-04-01_2013-08-31.json.gz=105B136Ysg9heeoOXpAwH9nSq8jsXZE4a
lens_lat_2012-09-01_2012-12-31.json.gz=104vZDKbZj1qXbnj-kwfN44y3OtyW-RFC
lens_lat_2012-04-01_2012-08-31.json.gz=101yBZSsTbiTo1vsma9TSU5XP8ugIFJip
lens_lat_2012-01-01_2012-03-31.json.gz=100Syzm4xJ9MC1SX2poTRXJjlbDyOoXSU
lens_lat_2011-09-01_2011-12-31.json.gz=10-lPswtacMlE98LN97eB4AbFJpWfpn63
lens_lat_2011-03-01_2011-08-31.json.gz=10-Tv-ULF0cgZoYyEMWoKKU4SoybYF5vl
lens_lat_2010-12-01_2011-02-28.json.gz=1-z7OftzGVh1tJyxK0M7d8icaJ4ERXuN1
lens_lat_2010-05-30_2010-11-30.json.gz=1-vIwF0yf4QlLbuhUapGgC-mIAe_uKmiW
lens_lat_2010-01-01_2010-04-30.json.gz=1-vFxXE2PZ4E1OJFr0EdE9H00IciUqq5I
lens_lat_2009-06-01_2009-12-31.json.gz=1-qgY9IVh2rzHFVxuD_YjQ0FHeDFcZbJo
lens_lat_2009-01-01_2009-05-31.json.gz=1-gl1tiVLNjiKQvZRqp7JVVzTZhH8oSvL
lens_lat_2008-06-01_2008-12-31.json.gz=1-dd6RoLePalsrxcVP9VGW1jwcmxhKJ1J
lens_lat_2008-01-01_2008-05-31.json.gz=1-ayRWDL8qkmZ2-MGDL7N-PDgkTNrfFih
lens_lat_2007-03-31_2007-12-31.json.gz=1-WMrYIl5GwP84UIacFD5z4tnyGyKDpAI
lens_lat_2006-08-01_2007-02-28.json.gz=1-WLO9AqMvNhrJ0WpkwNk8yXtEasNHdEj
lens_lat_2005-12-01_2006-07-31.json.gz=1-Vgr695qvfp7Kp8ED_DrQduPAJKuOxR_
lens_lat_2005-01-01_2005-11-30.json.gz=1-PLDU4UJziErzwaUHrrN3qF4QBymlP89
lens_lat_2004-01-01_2004-12-31.json.gz=1-MaQqD_OTNwIDWpgexqzwL2UGwHoOqfa
lens_lat_2003-01-01_2003-12-31.json.gz=1-G9SSaBb4FZeyvrZ7udemctcIrKdhyVm
lens_lat_2002-01-01_2002-12-32.json.gz=1-DGJTJzzgi4wyJwZkw0SHKOJ9LScD-Xo
lens_lat_2000-03-31_2001-12-31.json.gz=1-8cKkHJLofb8RpRdillKnVNDShrUqu-l
lens_lat_1998-05-31_2000-03-31.json.gz=1-8UrsABcPc2q2mLUhQtlq6RqThvGWzeV
lens_lat_1995-12-01_1998-05-31.json.gz=1-3UxBNSwM8D-ZKDpAN0Rya3Zdzjpxg3j
lens_lat_1991-02-01_1995-11-30.json.gz=180CBTphUQVwFIrfU3eW4kqd1F9q8R5XU
lens_lat_1984-01-01_1991-01-31.json.gz=11QhtKYIR8hXlEBBfdIVcvxPyC5tQrzAp
lens_lat_1900-01-01_1983-12-31.json.gz=11PnX2Z71bCJahtv-5uOaelAorkQMGe7l

Overwriting drive.cfg


In [ ]:
import wosplus as wp

In [ ]:
df=wp.wosplus('drive.cfg')

In [ ]:
j=44
ln=df.read_drive_json(jz[j]+'.gz',compression='gzip')#,encoding='latin1')

In [ ]:
ln.to_json('kk.json',lines=True,orient='records')

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',200)

In [ ]:
ln=dd.read_json('good.json',blocksize=100e6)

In [ ]:
ln.compute().shape

(190, 23)

In [ ]:
import sys

In [ ]:
ln=dd.read_json('bad.json',blocksize=100e7)

In [ ]:
try:
    print( ln.compute().shape )
except:
    sys.exit('Failed to load')

SystemExit: Failed to load

In [ ]:
ln=dd.read_json('kkk.json',blocksize=100e6)

In [ ]:
ln.compute().shape

(1, 23)